# **Part I: Using CNN**

Importing libraries

In [ ]:
from keras.datasets import fashion_mnist

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split


import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential, Input, Model 
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.layers import BatchNormalization

from sklearn.metrics import classification_report

import itertools

##**Data preparation**

###**Loading data set using libraries**



In [ ]:
(train_X,train_Y), (test_X,test_Y) = fashion_mnist.load_data()

### **Print size of train and test sets**

In [ ]:
print('Training data shape: ', train_X.shape, train_Y.shape)
print('Testing data shape: ', test_X.shape, test_Y.shape)

### **Count number of classes of data set**

In [ ]:
classes = np.unique(train_Y)
nclasses = len(classes)
print('Total number of outputs: ', nclasses)
print('Outputnclasses: ', classes)

### **Show two figures of dataset** (one of them from train set and another one from test set)

In [ ]:
plt.figure(figsize=[5,5])

# Display the first image in training data
plt.subplot(121)
plt.imshow(train_X[0,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(train_Y[0]))

# Display the first image in testing data
plt.subplot(122)
plt.imshow(test_X[0,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(test_Y[0]))

### **Reshape train and test set**

In [ ]:
train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)
train_X.shape, test_X.shape

### **Convert types of train and test sets**

In [ ]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255
test_X = test_X / 255

### **Convert class vectors to binary class matrix** for using categorical cross entropy as loss function in our CNN

In [ ]:
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)
print('Original label:', train_Y[0])
print('After conversion to one-hot', train_Y_one_hot[0])

## **Test set processing**

### **Split test set into two parts**: 1. test 2.valid




In [ ]:
test_x, valid_x, test_label, valid_label = train_test_split(test_X, test_Y_one_hot, test_size = 0.5, random_state = 1)

### **Show sizes of valid and test sets**

In [ ]:
test_x.shape, valid_x.shape, test_label.shape, valid_label.shape

## **First part for showing events underfit and overfit**

### **Initialize size of batch, number of epochs, and number of classes** for using in our CNN

In [ ]:
batch_size = 64
epochs = 20
num_classes = 10

### **Create our CNN  model**

In [ ]:
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size = (3, 3), activation = 'linear', input_shape=(28,28,1), padding = 'same'))
fashion_model.add(LeakyReLU(alpha = 0.1))
fashion_model.add(MaxPooling2D((2,2), padding = 'same'))

fashion_model.add(Conv2D(64, (3, 3), activation = 'linear', padding = 'same'))
fashion_model.add(LeakyReLU(alpha = 0.1))
fashion_model.add(MaxPooling2D(pool_size = (3,3), padding = 'same'))

fashion_model.add(Conv2D(128, (3, 3), activation = 'linear', padding = 'same'))
fashion_model.add(LeakyReLU(alpha = 0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))

fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation = 'linear'))
fashion_model.add(LeakyReLU(alpha = 0.1))                  
fashion_model.add(Dense(num_classes, activation = 'softmax'))

### **Compile our CNN model**

In [ ]:
fashion_model.compile(loss = keras.losses.categorical_crossentropy, optimizer = 'adam', metrics = ['accuracy'])

### **Summary of our CNN model**

In [ ]:
fashion_model.summary()

### **Teach our CNN model using train and valid sets**

In [ ]:
fashion_train = fashion_model.fit(train_X, train_Y_one_hot, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_x, valid_label))

### **Evaluate our CNN model on test set**(5000 samples)

In [ ]:
test_eval = fashion_model.evaluate(test_x, test_label, verbose = 0)

In [ ]:
print('Test loss: ', test_eval[0])
print('Test accuracy: ', test_eval[1])

### **Analyze the performance of our CNN model on train and valid sets**

اHere, We have both the event of overfit and underfit because
1. In first chart, our CNN model works better on train set than valid set. 
2. In second chart, our CNN model works better on valid set than train set.

In [ ]:
accuracy = fashion_train.history['accuracy']
val_accuracy = fashion_train.history['val_accuracy']
loss = fashion_train.history['loss']
val_loss = fashion_train.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label = 'Validation accuracy')
plt.title('Trainimg and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## **Second part with underfit and overfit events**

### **Initialize size of batch, number of epochs, and number of classes** for using in our CNN

In [ ]:
batch_size = 64
epochs = 20
num_classes = 10

### **Create our CNN  model**

In [ ]:
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size = (3, 3), activation = 'linear', input_shape=(28,28,1), padding = 'same'))
fashion_model.add(LeakyReLU(alpha = 0.1))
fashion_model.add(MaxPooling2D((2,2), padding = 'same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(64, (3, 3), activation = 'linear', padding = 'same'))
fashion_model.add(LeakyReLU(alpha = 0.1))
fashion_model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(128, (3, 3), activation = 'linear', padding = 'same'))
fashion_model.add(LeakyReLU(alpha = 0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))
fashion_model.add(Dropout(0.4))

fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation = 'linear'))
fashion_model.add(LeakyReLU(alpha = 0.1)) 
fashion_model.add(Dropout(0.3))
fashion_model.add(Dense(num_classes, activation = 'softmax'))

### **Summary of our CNN model**

In [ ]:
fashion_model.summary()

### **Compile our CNN model**

In [ ]:
fashion_model.compile(loss = keras.losses.categorical_crossentropy, optimizer = 'adam', metrics = ['accuracy'])

### **Teach our CNN model using train and valid sets**

In [ ]:
fashion_train_dropout = fashion_model.fit(train_X, train_Y_one_hot, batch_size = batch_size, epochs = epochs, verbose = 1, validation_data = (valid_x, valid_label))

### **Save our model** for using on test set(5000 samples)

In [ ]:
fashion_model.save("fashion_model_dropout.h5py")

### **Test our CNN model on test set(5000 samples)**

In [ ]:
test_eval = fashion_model.evaluate(test_x, test_label, verbose = 1)

In [ ]:
print('Test loss: ', test_eval[0])
print('Test accuracy: ', test_eval[1])

### **Analyze the performance of our CNN model on train and valid sets**

اHere, We don't have neither the event of overfit nor underfit because our CNN model works on train set as well as valid set. 

In [ ]:
accuracy = fashion_train_dropout.history['accuracy']
val_accuracy = fashion_train_dropout.history['val_accuracy']
loss = fashion_train_dropout.history['loss']
val_loss = fashion_train_dropout.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

### **Test our second CNN model on test set(5000 samples)**

In [ ]:
predictes_classes = fashion_model.predict(test_x)

In [ ]:
predicted_classes = np.argmax(np.round(predictes_classes), axis = 1)

In [ ]:
test_label_correct = np.argmax(np.round(test_label), axis = 1)

### **Show numbers of correct classification**
We show some images as sample.

In [ ]:
correct = np.where(predicted_classes == test_label_correct)[0]
print ("Found %d correct labels" % len(correct))
for i, correct in enumerate(correct[:9]):
    plt.subplot(3, 3, i + 1)
    plt.imshow(test_X[correct].reshape(28, 28), cmap = 'gray', interpolation = 'none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], test_label_correct[correct]))    
    plt.tight_layout()

### **Show numbers of incorrect classification**
We show some images as sample.

In [ ]:
incorrect = np.where(predicted_classes!=test_label_correct)[0]
print ("Found %d incorrect labels" % len(incorrect))
for i, incorrect in enumerate(incorrect[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], test_Y[incorrect]))
    plt.tight_layout()

### **Final evaluation of our second CNN model**

In [ ]:
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(test_label_correct, predicted_classes, target_names=target_names))